# Preprocessing

In [1]:
import librosa
import numpy as np

def preprocess_audio(audio_path, sample_rate=16000):
    """
    Vorverarbeitung einer Audiodatei: Resampling und Normalisierung.

    Args:
        audio_path (str): Pfad zur Audiodatei.
        sample_rate (int): Ziel-Sampling-Rate (Standard: 16000).

    Returns:
        np.array: Das vorverarbeitete Audio-Signal.
    """
    # Audiodatei laden und resamplen
    audio_signal, _ = librosa.load(audio_path, sr=sample_rate)

    # Zu Mono konvertieren (falls mehrkanalig)
    audio_signal = librosa.to_mono(audio_signal)

    # Normalisieren
    max_val = np.max(np.abs(audio_signal))
    if max_val > 0:
        audio_signal = audio_signal / max_val

    return audio_signal

# Funktion phoneme entropy

In [21]:
import numpy as np
import librosa

def calculate_phoneme_entropy(audio_signal, sample_rate=16000, frame_length=2048, hop_length=512):
    """
    Berechnet die Phoneme Entropy eines Audiosignals basierend auf der spektralen Energieverteilung.

    Quelle:
        Jelinek, F. (1997). Statistical Methods for Speech Recognition.
        MIT Press.

    Args:
        audio_signal (np.array): Das normalisierte Audio-Signal (1D-Array).
        sample_rate (int): Sampling-Rate des Signals (Standard: 16000 Hz).
        frame_length (int): Länge eines FFT-Frames in Samples (Standard: 2048).
        hop_length (int): Schrittweite zwischen Frames in Samples (Standard: 512).

    Returns:
        float: Phoneme Entropy-Wert für das gesamte Signal.
    """
    # Falls das Signal zu leise oder konstant ist, Entropie auf 0 setzen
    if np.all(audio_signal == audio_signal[0]):
        return 0.0

    # Berechnung des Leistungsverhältnisses (Power Spectrogram)
    spectrogram = np.abs(librosa.stft(audio_signal, n_fft=frame_length, hop_length=hop_length)) ** 2

    # Normalisierung über die Frequenzbänder (damit die Summe 1 ergibt)
    prob_distribution = spectrogram / np.sum(spectrogram, axis=0, keepdims=True)

    # Berechnung der Entropie pro Frame (log2, um Bits als Einheit zu haben)
    entropy_per_frame = -np.sum(prob_distribution * np.log2(prob_distribution + 1e-10), axis=0)

    # Mittelwert über alle Frames
    entropy_value = np.mean(entropy_per_frame)

    return entropy_value

# Test

In [22]:
import os

# Phoneme Entropy für alle Dateien berechnen
for file_name in sorted(os.listdir("../audio_files")):  # Alphabetische Sortierung
    if file_name.endswith(".wav"):
        file_path = os.path.join("../audio_files", file_name)

        # Audiodatei vorverarbeiten
        audio_signal = preprocess_audio(file_path)

        # Phoneme Entropy berechnen
        entropy_value = calculate_phoneme_entropy(audio_signal)

        print(f"Phoneme Entropy ({file_name}): {entropy_value:.4f}")

Phoneme Entropy (_noise_pink.wav): 6.8861
Phoneme Entropy (_noise_white.wav): 9.3703
Phoneme Entropy (_signal_constant.wav): 0.0000
Phoneme Entropy (_signal_silence.wav): 0.0000
Phoneme Entropy (_signal_sine.wav): 1.3561
Phoneme Entropy (_snr_03.wav): 5.6221
Phoneme Entropy (_snr_10.wav): 5.4378
Phoneme Entropy (_snr_20.wav): 5.0564
Phoneme Entropy (example1.wav): 3.9807
Phoneme Entropy (example2.wav): 3.9538
Phoneme Entropy (example3.wav): 4.4790
